In [ ]:
import functools
import pathlib

import numpy as np
import matplotlib.pyplot as plt

import shapely.geometry
import skimage.draw

import tensorflow as tf

import pydicom

import pymedphys
import pymedphys._dicom.structure as dcm_struct

In [ ]:
data_path_root = pathlib.Path.home().joinpath('.data/dicom-ct-and-structures')
dcm_paths = list(data_path_root.rglob('**/*.dcm'))
# dcm_paths

In [ ]:
structure_aliases = {
    "patient": "patient",
    "Patient": "patient",
    "PATIENT": "patient",
    "Number1": "remove",
    "Number2": "remove",
    "Number3": "remove",
    "Number4": "remove",
    "ANT Box": "box"
}

In [ ]:
structures_to_collect = ["patient", "box"]
slice_remove_name = "remove"

In [ ]:
dcm_headers = []

for dcm_path in dcm_paths:
    dcm_headers.append(pydicom.read_file(dcm_path, force=True, specific_tags=['SOPInstanceUID', 'SOPClassUID']))

In [ ]:
ct_image_paths = {
    header.SOPInstanceUID: path
    for header, path in zip(dcm_headers, dcm_paths)
    if header.SOPClassUID.name == "CT Image Storage"
}

In [ ]:
structure_set_paths = {
    header.SOPInstanceUID: path
    for header, path in zip(dcm_headers, dcm_paths)
    if header.SOPClassUID.name == "RT Structure Set Storage"
}

In [ ]:
names = set()

for uid, path in structure_set_paths.items():
    dcm = pydicom.read_file(path, force=True, specific_tags=['StructureSetROISequence'])
    for item in dcm.StructureSetROISequence:
        names.add(item.ROIName)

In [ ]:
# names

In [ ]:
names_map = {
    'BB': "bite_block",
    'Bladder': "bladder",
    "Bladder_obj": None,
    "Bowel": 'bowel',
    "Bowel_obj": None,
    "Box Adapter": None,
    "BoxAdaptor": None,
    "Brain": "brain",
    "Brainstem": "brainstem",
    "Bulla Lt": "bulla_left",
    "Bulla Rt": "bulla_right",
    "CORDprv": None,
    "CTV": None,
    "CTV Eval": None,
    "CTV thyroids": None,
    "CTVCT": None,
    "CTVMRI": None,
    "CTVSmall": None,
    "CTVeval": None,
    "CTVnew": None,
    "Chiasm": "chiasm",
    "Colon": "colon",
    "Colon_obj": None,
    "Cord": "cord",
    "Cord PRV": None,
    "Couch Edge": None,
    "Couch Foam Half Couch": None,
    "Couch Outer Half Couch": None,
    "GTV": None,
    "24.000Gy": None,
    "AdrenalGTV": None,
    "Bone_or": None,
    "BrainObj": None,
    "CTV1": None,
    "CTV_LN": None,
    "CTV_obj": None,
    "CTV_uncropped": None,
    "CTVmargin": None,
    "CTVmargin_eval": None,
    "CTVobj": None,
    "CTVobjnew": None,
    "CTVoptimise": None,
    "CTVoptimisenew": None,
    "Cauda equina": "causa_equina",
    "GTV LN": None,
    "GTV thyroids": None,
    "GTV+SCAR": None,
    "GTV-2": None,
    "OD": "lens_right",
    "OD Lens": "lens_right",
    "Lens OD": "lens_right",
    "OS": "lens_left",
    "OS lens": "lens_left",
    "Lens OS": "lens_left",
    
}

In [ ]:
mapped_names = set(names_map.keys())
print(mapped_names.difference(names))
names.difference(mapped_names)

In [ ]:
structure_set_path = list(structure_set_paths.items())[0][1]
structure_set_path

structure_set = pydicom.read_file(structure_set_path, force=True, specific_tags=['ROIContourSequence', 'StructureSetROISequence'])

In [ ]:
number_to_name_map = {
    roi_sequence_item.ROINumber: structure_aliases[roi_sequence_item.ROIName]
    for roi_sequence_item in structure_set.StructureSetROISequence
    if roi_sequence_item.ROIName in structure_aliases.keys()
}

In [ ]:
number_to_name_map

In [ ]:

single_contour_dataset = structure_set.ROIContourSequence[0].ContourSequence[0]

single_contour_dataset.ContourImageSequence[0].ReferencedSOPInstanceUID
single_contour_dataset.ContourData

In [ ]:
structure_set.ROIContourSequence[0].ReferencedROINumber

In [ ]:
structure_set.StructureSetROISequence[0]

In [ ]:
number_to_name_map = {
    roi_sequence_item.ROINumber: roi_sequence_item.ROIName
    for roi_sequence_item in structure_set.StructureSetROISequence
}

number_to_name_map

In [ ]:
contour_to_ct_map = {}

for uid, structure_set in structure_sets.items():
    structure_set.ROIContourSequence

In [ ]:
ct_to_structure_mapping = {}

for key in ct_image_paths.keys():
    

In [ ]:
# structure_sets

In [ ]:
a_ct_path = list(ct_image_paths.items())[100][1]
dcm = pydicom.read_file(a_ct_path, force=True)

In [ ]:
dcm.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
pixel_array = dcm.pixel_array

In [ ]:
plt.contourf(pixel_array)
plt.axis('equal')

In [ ]:
np.shape(pixel_array)

In [ ]:
tf.data.Dataset.from_tensor_slices([pixel_array])

In [ ]:
structure_sets

In [ ]:
dcm[-1].SOPClassUID.name == "CT Image Storage"

In [ ]:
dcm[-1].SOPClassUID.name

In [ ]:
dcm[0]

In [ ]:
dcm_path

In [ ]:
dcm_path

In [ ]:
dcm

In [ ]:
dcm_struct.list_structures(dcm)

In [ ]:
i = 7
x, y, z = dcm_struct.pull_structure('ANT Box', dcm)

x = x[i]
y = y[i]

In [ ]:
plt.plot(x, y)
plt.axis('equal')

In [ ]:
dx, dy = 2, 2
Cx, Cy = -100, -300

Ox, Oy = 1, 1

In [ ]:
r = (y - Cy) / dy * Oy
c = (x - Cx) / dx * Ox

In [ ]:
r

In [ ]:
np.array(list(zip(r*4, c*4)))

In [ ]:
img_size = 128
expansion = 4

In [ ]:
expanded_mask = skimage.draw.polygon2mask((img_size * expansion, img_size * expansion), np.array(list(zip(r*expansion, c*expansion))))

In [ ]:
plt.pcolormesh(expanded_mask)

In [ ]:
def reduce_expanded_mask(expanded_mask, img_size, expansion):
    expanded_mask = tf.dtypes.cast(expanded_mask, tf.float32)
    return tf.reduce_mean(
        tf.reduce_mean(
            tf.reshape(expanded_mask, (img_size, expansion, img_size, expansion)),
            axis=1,
        ),
        axis=2,
    )

In [ ]:
mask = reduce_expanded_mask(expanded_mask, img_size, expansion)

In [ ]:
plt.pcolormesh(mask)